In [1]:
import findspark
import time

from pyspark import SparkFiles
from pyspark.sql import SparkSession

findspark.init()
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [2]:
# Read the AWS S3 bucket into a DataFrame
url = 'https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv'
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)

In [3]:
# Create a temporary view of the DataFrame
home_sales_df.createOrReplaceTempView('home_sales')

In [4]:
# Calculate the average price for a four bedroom house sold per year, rounded to two decimal places
spark.sql('''
SELECT YEAR(date) AS sale_year, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY Sale_Year
ORDER BY Sale_Year
''').show()

+---------+-------------+
|sale_year|average_price|
+---------+-------------+
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [5]:
# Calculate the average price of a home for each year the home was built, that have 3 bedrooms and 3 bathrooms, rounded to two decimal places
spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
''').show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [6]:
# Calculate the average price of a home for each year the home was built, that have 3 bedrooms, 3 bathrooms, 
# with two floors, and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
''').show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [7]:
# Calculate the average price of a home per "view" rating, rounded to two decimal places, having 
# an average home price greater than or equal to $350,000. Order by descending view rating. 
# Determine the run time for this query.

def view_rating_query(temp_view):
    spark.sql(f'''
        SELECT view, ROUND(AVG(price), 2) AS average_price
        FROM {temp_view}
        GROUP BY view
        HAVING average_price >= 350000
        ORDER BY INT(view) DESC
    ''').show()

start_time = time.time()
view_rating_query('home_sales')
print("--- Query run time: %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- Query run time: 0.5395584106445312 seconds ---


In [8]:
# Cache the the temporary view
spark.sql('CACHE TABLE home_sales')
spark.catalog.isCached('home_sales')

True

In [9]:
# Repeat the most recent query using the cached data. Determine the runtime and compare it to the uncached runtime.
start_time = time.time()
view_rating_query('home_sales')
print("--- Cached query run time: %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- Cached query run time: 0.35955023765563965 seconds ---


In [10]:
# Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy('date_built').mode('overwrite').parquet('home_sales_partitioned')
# Read the formatted parquet data
partitioned_home_sales_df = spark.read.parquet('home_sales_partitioned')
# Create a temporary view for the parquet data
partitioned_home_sales_df.createOrReplaceTempView('home_sales_partitioned_view')

In [11]:
# Repeat the most recent query using the parquet DataFrame. Determine the runtime and compare it to the uncached runtime.
start_time = time.time()
view_rating_query('home_sales_partitioned_view')
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.8302958011627197 seconds ---


In [12]:
# Uncache the home_sales temporary table
spark.sql('UNCACHE TABLE home_sales')
spark.catalog.isCached('home_sales')

False